In [1]:
import cv2

In [2]:
# YOLO modelini yükle
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
with open("walking.mp4", "r", encoding="latin-1") as f:
    classes = f.read().strip().split("\n")

layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# Video akışını aç
cap = cv2.VideoCapture("x2mate.com-Okulöncesi .mp4") 

while True:
    ret, frame = cap.read()
    height, width, channels = frame.shape

    # YOLO için görüntüyü hazırla
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    if len (outs) == 0:
        print("No Detection Found")
        continue

    # Tespit edilen nesneleri çerçeve üzerinde işaretle
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
            # Tespit edilen nesnenin merkezi koordinatlarını ve boyutlarını al
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

            # Dikdörtgenin köşe koordinatlarını hesapla
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

        if len(boxes) == 0 or len(class_ids) == 0 or len(confidences) == 0:
            print("No objects detected or empty lists")
            continue
    # Non-maximum suppression uygula
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Tespit edilen nesneleri çerçeve üzerinde işaretle
    for i in range(len(boxes)):
        if i in indexes and len(class_ids) > i:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = (0, 255, 0)  # Yeşil renk
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, f"{label} {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Sonuçları göster
    cv2.imshow("YOLO Object Detection", frame)

    # 'q' tuşuna basıldığında döngüden çık
    if cv2.waitKey(1) & 0xFF== ord("q"):
        break

# Video akışını serbest bırak ve pencereyi kapat
cap.release()
cv2.destroyAllWindows()

IndexError: invalid index to scalar variable.